In [ ]:
# importing packages
import pandas as pd
import numpy as np
import binascii
import crcmod
import random

In [ ]:
# google drive documents
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importing washuHistory text file (the file we have to send)

# opening and reading the Washu History txt file
washuHist = open('drive/MyDrive/ESE572/Project2/washuHistory.txt', 'r')
# string version of the Washu History text file
strWashu = washuHist.read()
print(strWashu)

In 1853, prominent St. Louis merchant Wayman Crow and his pastor, William Greenleaf Eliot Jr., concerned about the lack of institutions of higher learning in the growing midwest, led the founding of Washington University in St. Louis. During the 1840s and 50s, waves of immigrants flooded into St. Louis, boosting the population of the young city. With these newcomers came a pressing need for education — both industrial training and basic general courses — conducted outside of normal working hours. So the first educational step of the young Washington University was to establish an evening program on October 22, 1854. Over the succeeding decades, the continuing education program underwent many changes. The university flourished at its location in downtown St. Louis for its first 50 years, growing from an evening program to an institution offering a full slate of scientific, liberal arts and classical course offerings. In time, schools of law and fine arts were added. In 1891, the school 

In [ ]:
# Testing Ascii conversion
testStr = 'In18'
testStrArr = [x for x in testStr]
testAscii = []
for i in testStr:
  testAscii.append(format(ord(i), '08b'))
# testAscii = ''.join(format(ord(i), '08b') for i in testStr)
print(testStrArr)
print(testAscii) 
test_binary = ''.join(testAscii)
print(test_binary)

['I', 'n', '1', '8']
['01001001', '01101110', '00110001', '00111000']
01001001011011100011000100111000


In [ ]:
# Step 1 Convert Washu History document from string to binary

# asciiWashu contains the binary conversion of the text document using the ascii table
asciiWashu = ''.join(format(ord(i), '08b') for i in strWashu) 
print(asciiWashu)
# checking if the conversion was done correctly
asciiCheck = []
for i in strWashu:
  asciiCheck.append(format(ord(i), '08b'))
print(len(asciiCheck))
print(asciiCheck)
# conversion is done correctly because length of array (individual conversions for each character in the text including spaces) is the same as the character count in the original word document

0100100101101110001000000011000100111000001101010011001100101100001000000111000001110010011011110110110101101001011011100110010101101110011101000010000001010011011101000010111000100000010011000110111101110101011010010111001100100000011011010110010101110010011000110110100001100001011011100111010000100000010101110110000101111001011011010110000101101110001000000100001101110010011011110111011100100000011000010110111001100100001000000110100001101001011100110010000001110000011000010111001101110100011011110111001000101100001000000101011101101001011011000110110001101001011000010110110100100000010001110111001001100101011001010110111001101100011001010110000101100110001000000100010101101100011010010110111101110100001000000100101001110010001011100010110000100000011000110110111101101110011000110110010101110010011011100110010101100100001000000110000101100010011011110111010101110100001000000111010001101000011001010010000001101100011000010110001101101011001000000110111101100110001000000110100101101110

In [ ]:
# Step 2 functions used to computing the final CRC 16 bits

# crc_remainder calcutes the 16 bit remainder
# set initial_filler to 0
def crc_remainder(input_bitstring, polynomial_bitstring, initial_filler):
    #Calculate the CRC remainder of a string of bits using a chosen polynomial. initial_filler should be '1' or '0'
    polynomial_bitstring = polynomial_bitstring.lstrip('0')
    len_input = len(input_bitstring)
    initial_padding = (len(polynomial_bitstring) - 1) * initial_filler
    input_padded_array = list(input_bitstring + initial_padding)
    while '1' in input_padded_array[:len_input]:
        cur_shift = input_padded_array.index('1')
        for i in range(len(polynomial_bitstring)):
            input_padded_array[cur_shift + i] \
            = str(int(polynomial_bitstring[i] != input_padded_array[cur_shift + i]))
    return ''.join(input_padded_array)[len_input:]

# crc_check checks if a CRC value is a valid CRC for the inputted bitstring
# if the CRC value is valid, the function will return True, otherwise return False
def crc_check(input_bitstring, polynomial_bitstring, check_value):
    # Calculate the CRC check of a string of bits using a chosen polynomial
    polynomial_bitstring = polynomial_bitstring.lstrip('0')
    len_input = len(input_bitstring)
    initial_padding = check_value
    input_padded_array = list(input_bitstring + initial_padding)
    while '1' in input_padded_array[:len_input]:
        cur_shift = input_padded_array.index('1')
        for i in range(len(polynomial_bitstring)):
            input_padded_array[cur_shift + i] \
            = str(int(polynomial_bitstring[i] != input_padded_array[cur_shift + i]))
    return ('1' not in ''.join(input_padded_array)[len_input:])


In [ ]:
# Step 2 Creating 216 bit segments by dividing binary file into 200 bits + 16 bits from CRC function result using CRC 16 function X16 + X12 + X5 + 1 or 10001000000100001

# Splitting binary into 200 bit frames
# allFrame200 is an array that contains the 200 bit frames from the original ascii binary
frame200 = []
# counter for counting to 200
n = 0
# base frame for creating string of 200 bits
frame = ''
for i, bit in enumerate(asciiWashu):
  # if n does not equal 200 add another bit into the frame
  frame = ''.join([frame, str(bit)])
  n += 1
  # if n equals 200 or at the end of the ascii binary bits, append the finished frame to the 200 bit array and reset frame to blank
  if n >= 200 or i == len(asciiWashu)-1:
    frame200.append(frame)
    frame = ''
    n = 0

# 200 bit with CRC added on array
frameCRC = []
# CRC 16 function
CRC16func = '10001000000100001'

# CRC array containing CRC 16 bit results
CRCarr = []
for i in range(len(frame200)):
  CRCarr.append(crc_remainder(frame200[i],CRC16func,'0'))
  frameCRC.append(''.join([frame200[i],CRCarr[i]]))



In [ ]:
# Step 3 apply FEC algorithm to the 216 bit segments

# applyFEC takes a datasegment and then triples the quantity of each entry
# for example if inputed '0', output is '000'
def applyFEC(dataSegment):
  FECarr = []
  for bit in dataSegment:
    FECarr.append(''.join([bit,bit,bit]))
  return ''.join(FECarr)

# Apply the FEC to the 216 bit segment to create a 648 segment for each frame
frameFEC = []
for i in range(len(frameCRC)):
  frameFEC.append(applyFEC(frameCRC[i]))


In [ ]:
# Step 4 Inserting the framing bit and bit stuffing

# bit stuffing the 648 bit segments
def bitStuff(dataSegment):
  dataList = []
  movingSeg = []
  for i, bit in enumerate(dataSegment):
    dataList.append(bit)
    if i < 5:
      movingSeg.append(int(bit))

  # moving the window through the whole data segment
  a = 5
  b = len(dataList)
  while a < b:
    if sum(movingSeg) > 4:
      dataList.insert(a,'0')
    movingSeg = movingSeg[1:]
    movingSeg.append(int(dataList[a]))
    a += 1
    b = len(dataList)
  # checking last window if there are 5 ones in a row
  if sum(movingSeg) > 4:
    dataList.append('0')
  # turning the list back to strings from ints
  dataList = [str(num) for num in dataList]
  # return the joined string from the list
  return ''.join(dataList)

# creating array to store bitStuffed segments
frameBitStuffed = []
for i in range(len(frameFEC)):
  frameBitStuffed.append(bitStuff(frameFEC[i]))

# Flag pattern 0,1^6,0
flag = '01111110'
# creating array to store framed segments with flag
frameFlag = []
for i in range(len(frameBitStuffed)):
  frameFlag.append(''.join([flag,frameBitStuffed[i]]))
  if i == len(frameBitStuffed) - 1:
    frameFlag[i] = ''.join([frameFlag[i],flag])

# creating transmitted datastream
transmitData = ''.join(frameFlag)
print(transmitData)


...


In [ ]:
# Step 5 Create errors in the transmitted frames and then correct using the FEC algorithm. Then check CRC for each frame and determine how many frames fail 

# the transmitted data is in the variable 'transmitData' which is a string of binary

# steps to process transmitted data

# initilization: get some general parameter in case of preparation

# apply random error to the data with 0.1 probability (10% errors) and 0.01 probability (1% errors)
def applyError(data, probability):
  # get the length of the data
  len_data = len(data)
  # calculate how many number do we need to simulate as error
  len_error = int(len_data*probability)
  # generate what index should be errored
  random.seed(10)
  idx_error = random.sample(range(0,len_data-1),len_error)
  # order the index
  idx_error.sort()
  # simulate the error data from 0 to 1 or from 1 to 0
  new_data = list(data);
  for i in idx_error:
    if new_data[i] == '1':
      new_data[i] = '0';
    elif new_data[i] == '0':
        new_data[i] = '1';
  return ''.join(new_data)
  
# break the data with error into frames using the flag pattern
def removeFlag(data):
  # get the length of all data
  len_data = len(data)
  # set the end index
  i_end = len_data-8
  # initilize the return data with a list of all frames
  new_data = []
  # set the number of flag to operation
  num = 0
  # create a window of length of 8bit and identify "01111110" 
  for i in range(0,i_end+1):
    if data[i:i+8] == '01111110':
      # set the start index of one frame
      if num == 0:
        f_start = i+8
        num += 1
      elif num > 0:
        new_data.append(data[f_start:i])
        f_start = i+8
        num += 1
  return new_data

# remove bit stuffing from each frame of the data
def removeBitStuff(frame):
  # convert string to list for deleting
  list_frame = []
  # set variable storing all index for deleting
  idx = []
  # create a window to identify if there is "111110", and remove the '0' if it is
  for i in frame:
    list_i = list(i)
    for j in range(0,len(i)-6):
      if i[j:j+6] == '111110':
        idx.append(j+5)
    for k in idx[::-1]:
      del list_i[k]
    list_frame.append(''.join(list_i))
    idx = []
  return list_frame

# use FEC algorithm to correct errors in each frame of the data
def checkFEC(frame):
# initilize the return list
  list_frame = []
  for i in frame:
    list_i = list(i)
    new_list = []
    # first identify if the frame can be divided by 3, if not, that shows the frame is wrong code
    if len(i) % 3 == 0:
      idx_end = int(len(i) / 3)
      for j in range(idx_end):
        if list_i[j*3:j*3+3].count('0') > 1:
          new_list.append('0')
        else:
          new_list.append('1')
      list_frame.append(''.join(new_list))
    else:
      print("False in dividable by 3")
      continue
  return list_frame

# frames should now be 216 bit segments lengths
# remove the last 16 bits of data on the end of the segments to produce original 200 bit segments
def removeLast16(frame):
  # initilize the return list and the iteration times
  list_frame = []
  num = 0
  for i in frame:
    num += 1
    # first identify if the length of frame is 216, it's wrong if it's not 216 bit
    if len(i) == 216 or num == len(frame):
      list_frame.append(i[0:len(i)-16])
    else:
      print("False in length of 216")
      continue
  return list_frame

# See if the CRC of the transmitted frame is valid and determine the number of frames that fail the CRC check
# the method 'crc_check(input_bitstring, polynomial_bitstring, check_value)' is already written in the Step 2 code
# you can use that method to check or write your own code for it
def failedCRC(allFrames):
  # find the corresponding FCS by checking every string of allFrames
  idx_allFrames = []
  idx_FCS = []
  # initilize the CRC check result
  CRC_matched = []
  CRC_failed_idx = []
  # get all index about which one string in allFrames corresponds to the same string in frame200
  for i in allFrames:
    # identify if the current string is in the original string
    if i in frame200:
      idx_FCS.append(frame200.index(i))
      idx_allFrames.append(allFrames.index(i))
  # check to see the CRC result
  for i,j in zip(idx_allFrames,idx_FCS):
    CRC_matched.append([i,j,crc_check(allFrames[i], CRC16func, CRCarr[j])])
  for i in range(len(CRCarr)):
    if i not in idx_FCS:
      CRC_failed_idx.append(i)
  return idx_allFrames,idx_FCS,CRC_failed_idx

# make sure to do the process with both of the probability errors



In [ ]:
prob = 0.1
right1, wrong1 = 0, 0
right2, wrong2 = 0, 0 
for i in range(len(frameFEC)):
  originalFEC = removeLast16(checkFEC([applyError(frameFEC[i], 0)]))
  errorFEC = removeLast16(checkFEC([applyError(frameFEC[i], prob)]))
  if i == 0:
    print(applyError(frameFEC[i], 0))
    print(applyError(frameFEC[i], prob))
    print(originalFEC)
    print(errorFEC)
  oriCRC = crc_remainder(originalFEC[0], CRC16func, '0')
  errorCRC = crc_remainder(errorFEC[0], CRC16func, '0')
  if oriCRC == errorCRC:
    right1 += 1
  else:
    wrong1 += 1
  check1 = crc_check(originalFEC[0], CRC16func, errorCRC)
  if check1:
    right2 += 1
  else:
    wrong2 += 1

ratio1 = right1 / (right1 + wrong1)
ratio2 = right2 / (right2 + wrong2)
print(ratio1, ratio2)


000111000000111000000111000111111000111111111000000000111000000000000000000000111111000000000111000000111111111000000000000000111111000111000111000000111111000000111111000000111000111111000000000000111000000000000000000111111111000000000000000111111111000000111000000111111000111111111111000111111000111111000111000111111000111000000111000111111000111111111000000111111000000111000111000111111000111111111000000111111111000111000000000000111000000000000000000111000111000000111111000111111111000111000000000000111000111111111000000000111000000000000000000111000000111111000000000111111000111111111111111111111000111000111111111111000111000111000000
000101000000111100000111000111110101101111111100000000111000000000100000010001111101000000000111000000111111111000000000000000111111000100000101000000111111000010110111000000111010111111000000000000101000000000010000000111111111010000000000010100111111000100111000000101111000111111110111001111111000111111001101000111111100111000001110000111111000111

In [ ]:
################################################################################
# DEFUNCT CODE BANK
# DON'T RUN CODE IN HERE
# JUST KEEPING IT HERE JUST IN CASE NEEDED AGAIN
################################################################################

# testing crcmod library to find crc
_CRC_FUNC = crcmod.mkCrcFun(0x11021, initCrc=0, xorOut=0xffff)
data = bytearray.fromhex("023b010000000000")
crc = _CRC_FUNC(data)

# testing string
strTest = 'Hi'

array1=np.random.randint(0,511, size=100000)
array2=np.random.randint(0,511, size=100000)
zb = np.zeros(1, dtype=np.uint8)[0].tobytes()
for j in range(10):
    crc=0xffff
    tmp_data = array1[j].tobytes()
    crc=binascii.crc_hqx(zb, crc)
    print(crc)
    crc=binascii.crc_hqx(tmp_data, crc)

    tmp_data = array2[j].tobytes()
    crc=binascii.crc_hqx(zb, crc)
    crc=binascii.crc_hqx(tmp_data, crc)

print(array1)
print(array2)
print(zb)

# Testing CRC function
print(allFrame200[0])
# print(binDiff(allFrame200[0]))

# bitDiff takes the input of 17 bits and takes the XOR operation with the CRC 16 bit function
# input: 17 bits
# output: XOR result (17 bits)
def bitDiff(bitSection):
  result = []
  CRCfunc = str(11000000000000101)
  crcList = list(CRCfunc)
  bitList = list(bitSection)
  for i in range(len(bitList)):
    result.append(str(abs(int(crcList[i]) - int(bitList[i]))))
  result = ''.join(result)
  return result

# creating array to store calculated CRC 16 bits
CRCarr = []
# using CRC function on all frames
for i in range(len(allFrame200)):
  # initialize first 17 bits to be divided by CRC function
  diff = ''
  # if the frame has less than 17 bits
  if len(allFrame200[i]) < 17:
    for l in range(len(allFrame200[i])):
      diff = ''.join([diff, allFrame200[i][l]])
    # add 0's onto the end the frame until there are 17 bits
    counter = len(diff)
    while counter < 17:
      diff = ''.join([diff, '0'])
      counter += 1
  # if frame has more than 17 bits
  else:
    for j, bit1 in enumerate(allFrame200[i]):
      if j > 16:
        break
      diff = ''.join([diff, bit1])
  
  # if the frame is 17 long
  if len(allFrame200[i]) <= 17:
    # take the XOR differences
    diff = bitDiff(diff)
    # # if the difference starts with a 1, do the operation again so that it starts with a 0
    # if diff[0] == '1':
    #   diff = bitDiff(diff)
    # # take 0 off the beginning until diff starts with 1
    # while diff[0] == '0':
    #   diff = diff[1:]
    # # add 0's onto the end of diff until the length is 16
    # while len(diff) < 16:
    #   diff = ''.join([diff, '0'])
      
    # take the first digit off of the 17 bits to make it 16 bit long
    print(diff)
    diff = diff[1:]
    print(diff)
  # if the frame is longer than 17 bits
  else:
    # if the frame is longer than 17 do the long division on all of the frame starting with the 17 bits
    k = 17
    while k < len(allFrame200[i]):
      # take the XOR difference
      diff = bitDiff(diff)
      # if the term starts with a '0'
      if diff[0] == '0':
        diff = ''.join([diff[1:], allFrame200[i][k]])
        k += 1
      # after last XOR division take the 0 off the result and the answer is remainder
      if k == 200:
        print(diff)
        diff = ''.join(diff[1:])
        print(diff)
      # if i == 0:
      #   print(str(11000000000000101), diff, k)
  CRCarr.append(diff)
print(CRCarr)

beforeError = frame200[0]
afterError = applyError(frame200[0], 0.01)
right, wrong = 0, 0
for i in range(len(beforeError)):
  if beforeError[i] == afterError[i]:
    right += 1
  else:
    wrong += 1
  
ratioRight = right / (right + wrong)
print(ratioRight)
    


In [ ]:
# run the code and return the result with probability 0.1 and 0.01
idx_allFrames_1,idx_matched_1,idx_failed_1 = failedCRC(removeLast16(checkFEC(removeBitStuff(removeFlag(applyError(transmitData, 0.1))))))

idx_allFrames_2,idx_matched_2,idx_failed_2 = failedCRC(removeLast16(checkFEC(removeBitStuff(removeFlag(applyError(transmitData, 0.01))))))

pd.set_option('display.max_colwidth',180)
d = {'probability':[0.1,0.01],'matched index': [idx_matched_1,idx_matched_2], 'failed index': [idx_failed_1,idx_failed_2]}
df = pd.DataFrame(data=d)
df

,probability,matched index,failed index
0,0.10,[],"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]"
1,0.01,"[8, 37, 38]","[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 39, 40, 41, 42, 43, 44, 45]"


In [ ]:
# set a frame to show the whole step
test_Str = 'in18'
test_binary = ''.join(testAscii)
test_crc_remainder = crc_remainder(test_binary,CRC16func,'0')
test_CRC = ''.join([test_binary,test_crc_remainder])
test_FEC = applyFEC(test_CRC)
test_bitStuffing = bitStuff(test_FEC)
flag = '01111110'
test_flag = flag + test_bitStuffing + flag
test_remove_flag = removeFlag(test_flag)
test_remove_bitStuff = removeBitStuff(test_remove_flag)
test_check_FEC = checkFEC(test_remove_bitStuff)
test_remove_CRC_remainder = removeLast16(test_check_FEC)
test_check_CRC = failedCRC(test_remove_CRC_remainder)


test_list = [test_Str,test_binary,test_CRC,test_FEC,test_bitStuffing,test_flag,test_remove_flag[0],test_remove_bitStuff[0],test_check_FEC[0],test_remove_CRC_remainder[0],"True"]
data = pd.DataFrame({'data step by step':test_list})
data.index = ['words','binary data','append CRC remainder','FEC algorithm','bit stuffing','flag','remove flag','remove bit stuffing','remove FEC', 'remove CRC remainder', 'check CRC']
data

,data step by step
words,in18
binary data,01001001011011100011000100111000
append CRC remainder,010010010110111000110001001110001001110001111110
FEC algorithm,000111000000111000000111000111111000111111111000000000111111000000000111000000111111111000000000111000000111111111000000000111111111111111111000
bit stuffing,00011100000011100000011100011111010001111101111000000000111110100000000011100000011111011110000000001110000001111101111000000000111110111110111110111000
flag,011111100001110000001110000001110001111101000111110111100000000011111010000000001110000001111101111000000000111000000111110111100000000011111011111011111011100001111110
remove flag,00011100000011100000011100011111010001111101111000000000111110100000000011100000011111011110000000001110000001111101111000000000111110111110111110111000
remove bit stuffing,000111000000111000000111000111111000111111111000000000111111000000000111000000111111111000000000111000000111111111000000000111111111111111111000
remove FEC,010010010110111000110001001110001001110001111110
remove CRC remainder,01001001011011100011000100111000
